In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, Input
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

from tensorflow.keras.optimizers import Adam

import numpy as np

In [3]:
def preprocess_input_grayscale(img):
    return np.repeat(img, 3, axis=-1)

In [4]:
train_datagen = ImageDataGenerator(
    # preprocessing_function=preprocess_input_grayscale,
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    shear_range=0.2,
    fill_mode='wrap',
)

train_generator = train_datagen.flow_from_directory(
    'Dataset_train',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
)

val_datagen = ImageDataGenerator(
    # preprocessing_function=preprocess_input_grayscale,
    rescale=1./255
)

val_generator = val_datagen.flow_from_directory(
    'Dataset_testing',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

Found 4444 images belonging to 8 classes.
Found 1114 images belonging to 8 classes.


In [5]:
num_classes = len(train_generator.class_indices)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(224,224,3)),
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax') #Output
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'precision', 'recall'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     1,311,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │         2,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,227,912 (16.13 MB)

 Trainable params: 1,969,928 (7.51 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [6]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=5,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    verbose=1
)

c:\Users\rizfi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 878ms/step - accuracy: 0.1969 - loss: 2.2368 - precision: 0.2189 - recall: 0.0282

c:\Users\rizfi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


139/139 ━━━━━━━━━━━━━━━━━━━━ 154s 1s/step - accuracy: 0.1971 - loss: 2.2357 - precision: 0.2191 - recall: 0.0282 - val_accuracy: 0.2828 - val_loss: 1.8737 - val_precision: 0.7500 - val_recall: 0.0054
Epoch 2/5


c:\Users\rizfi\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 3/5
139/139 ━━━━━━━━━━━━━━━━━━━━ 112s 803ms/step - accuracy: 0.2937 - loss: 1.8756 - precision: 0.4123 - recall: 0.0253 - val_accuracy: 0.3312 - val_loss: 1.8039 - val_precision: 0.7368 - val_recall: 0.0126
Epoch 4/5
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 347us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 5/5
139/139 ━━━━━━━━━━━━━━━━━━━━ 111s 800ms/step - accuracy: 0.3101 - loss: 1.8311 - precision: 0.4912 - recall: 0.0492 - val_accuracy: 0.3178 - val_loss: 1.7795 - val_precision: 0.7679 - val_recall: 0.0386


In [7]:
# Mendapatkan nilai akurasi untuk setiap epoch dari history
train_accuracy = history.history['accuracy']  # Akurasi pada data training
val_accuracy = history.history['val_accuracy']  # Akurasi pada data validasi

# Mencetak akurasi di setiap epoch
print("Training Accuracy per Epoch:")
for epoch, acc in enumerate(train_accuracy, 1):
    print(f"Epoch {epoch}: {acc:.4f}")

print("\nValidation Accuracy per Epoch:")
for epoch, val_acc in enumerate(val_accuracy, 1):
    print(f"Epoch {epoch}: {val_acc:.4f}")

Training Accuracy per Epoch:
Epoch 1: 0.2243
Epoch 2: 0.0000
Epoch 3: 0.2910
Epoch 4: 0.0000
Epoch 5: 0.3128

Validation Accuracy per Epoch:
Epoch 1: 0.2828
Epoch 2: 0.3312
Epoch 3: 0.3178


In [8]:
for layer in base_model.layers[:-20]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

fine_tune_history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=15,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    verbose=1
)

Epoch 1/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 344s 2s/step - accuracy: 0.2534 - loss: 1.8985 - val_accuracy: 0.3609 - val_loss: 1.7074
Epoch 2/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 323s 2s/step - accuracy: 0.3010 - loss: 1.8016 - val_accuracy: 0.3537 - val_loss: 1.7178
Epoch 4/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 848us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 306s 2s/step - accuracy: 0.3170 - loss: 1.7859 - val_accuracy: 0.3528 - val_loss: 1.7365
Epoch 6/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 401us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 306s 2s/step - accuracy: 0.3367 - loss: 1.7195 - val_accuracy: 0.3537 - val_loss: 1.7337
Epoch 8/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 299s 2s/step - accuracy: 0.3446 - loss: 1.7126 - val_accuracy:

In [9]:
# Mendapatkan nilai akurasi untuk setiap epoch dari history
train_accuracy = fine_tune_history.history['accuracy']  # Akurasi pada data training
val_accuracy = fine_tune_history.history['val_accuracy']  # Akurasi pada data validasi
# Mencetak akurasi di setiap epoch
print("Training Accuracy per Epoch:")
for epoch, acc in enumerate(train_accuracy, 1):
    print(f"Epoch {epoch}: {acc:.4f}")

print("\nValidation Accuracy per Epoch:")
for epoch, val_acc in enumerate(val_accuracy, 1):
    print(f"Epoch {epoch}: {val_acc:.4f}")

Training Accuracy per Epoch:
Epoch 1: 0.2626
Epoch 2: 0.0000
Epoch 3: 0.3092
Epoch 4: 0.0000
Epoch 5: 0.3261
Epoch 6: 0.0000
Epoch 7: 0.3348
Epoch 8: 0.0000
Epoch 9: 0.3515
Epoch 10: 0.0000
Epoch 11: 0.3645
Epoch 12: 0.0000
Epoch 13: 0.3704
Epoch 14: 0.0000
Epoch 15: 0.3803

Validation Accuracy per Epoch:
Epoch 1: 0.3609
Epoch 2: 0.3537
Epoch 3: 0.3528
Epoch 4: 0.3537
Epoch 5: 0.3734
Epoch 6: 0.3842
Epoch 7: 0.3923
Epoch 8: 0.4013
